In [2]:
import torch
import torch.nn as nn
from config import ModelConfig

batch_size = 2
seq_len = 4
dim = 64
n_head = 16
head_dim = dim // n_head
base = 10_000

In [3]:

q = torch.rand(batch_size,n_head,seq_len,head_dim)
k = torch.rand(batch_size,n_head,seq_len,head_dim)


In [6]:
q.float().view(*q.shape[:-1], -1, 2).shape

torch.Size([2, 16, 4, 2, 2])

In [7]:
q.shape

torch.Size([2, 16, 4, 4])

In [8]:
4*12*128

6144

In [13]:
import math
from dataclasses import dataclass
from typing import Tuple, Optional, Literal

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
import os

In [14]:
@dataclass
class ModelArgs:
    """
Model argümanlarını ve hiperparametreleri tanımlayan veri sınıfı.

Öznitelikler (Attributes):
    max_batch_size (int): Maksimum batch (yığın) boyutu.
    max_seq_len (int): Maksimum dizi (sequence) uzunluğu.
    dtype (Literal["bf16", "fp8"]): Hesaplamalar için kullanılacak veri tipi.
    vocab_size (int): Kelime dağarcığı (vocabulary) boyutu.
    dim (int): Modelin genel gizli katman boyutu (embedding + hidden dim).
    inter_dim (int): MLP (besleyici ağ) katmanları için ara katman boyutu.
    moe_inter_dim (int): MoE (Mixture of Experts) katmanları için ara katman boyutu.
    n_layers (int): Transformer katmanı sayısı.
    n_dense_layers (int): Modeldeki yoğun (dense) katman sayısı.
    n_heads (int): Dikkat (attention) başlığı sayısı.
    n_routed_experts (int): MoE içinde yönlendirilen uzman sayısı.
    n_shared_experts (int): MoE içinde paylaşılan (her gruba açık) uzman sayısı.
    n_activated_experts (int): Her örnek için aktif edilen uzman sayısı.
    n_expert_groups (int): Uzman grubu sayısı (MoE routing grupları).
    n_limited_groups (int): MoE yönlendirmesinde sınırlandırılmış grup sayısı.
    score_func (Literal["softmax", "sigmoid"]): MoE yönlendirme puanlama fonksiyonu.
    route_scale (float): Routing skorları için çarpan ölçekleme katsayısı.
    q_lora_rank (int): Query (sorgu) projeksiyonları için LoRA rank’ı.
    kv_lora_rank (int): Key-Value (anahtar-değer) projeksiyonları için LoRA rank’ı.
    qk_nope_head_dim (int): Konumsal bilgi olmadan QK projeksiyonları için başlık boyutu.
    qk_rope_head_dim (int): Rotary Positional Embedding kullanılan QK projeksiyon başlık boyutu.
    v_head_dim (int): Value (değer) projeksiyon başlık boyutu.
    original_seq_len (int): Modelin önceden eğitim aldığı maksimum dizgi uzunluğu.
    rope_theta (float): Rotary positional encoding için temel (üstel frekans) değeri.
    rope_factor (float): Rotary frekans düzeltmesi için ölçekleme katsayısı.
    beta_fast (int): Düşük rotasyon eşiği (erken düzeltme için).
    beta_slow (int): Yüksek rotasyon eşiği (tam düzeltme için).
    mscale (float): Uzatılmış dikkat (extended attention) için ölçekleme katsayısı.
    """
    max_batch_size: int = 8
    max_seq_len: int = 256
    dtype: Literal["bf16", "fp8"] = "bf16"
    vocab_size: int = 50256
    dim: int = 1024
    inter_dim: int = 4 * dim
    moe_inter_dim: int = 704
    n_layers: int = 6
    n_dense_layers: int = 1
    n_heads: int = 8
    # moe
    n_routed_experts: int = 8
    n_shared_experts: int = 2
    n_activated_experts: int = 4
    n_expert_groups: int = 1
    n_limited_groups: int = 1
    score_func: Literal["softmax", "sigmoid"] = "softmax"
    route_scale: float = 1.
    # mla
    q_lora_rank: int = 0
    kv_lora_rank: int = 256
    qk_nope_head_dim: int = 64
    qk_rope_head_dim: int = 32
    v_head_dim: int = 64
    # yarn
    original_seq_len: int = 512
    rope_theta: float = 10000.0
    rope_factor: float = 40
    beta_fast: int = 32
    beta_slow: int = 1
    mscale: float = 1.

    # data preparing
    shuffle: bool = True
    drop_last: bool = True

    # training
    train:bool = True
    dataset_path = "/kaggle/input/clenaned-pretrain-data/cleaned_pre-data_final.txt" if os.path.exists("/kaggle/input/clenaned-pretrain-data/cleaned_pre-data_final.txt") else "8k_data.txt"

def precompute_freqs_cis(args: ModelArgs) -> torch.Tensor:
    """
Rotary pozisyonel gömmeler (rotary positional embeddings) için frekansa dayalı kompleks üstel değerleri önceden hesaplar.

Parametreler (Args):
    args (ModelArgs): Pozisyonel gömme parametrelerini içeren model argümanları.

Dönüş (Returns):
    torch.Tensor: Pozisyonlara karşılık gelen karmaşık (complex) üstel değerleri içeren bir tensor.
    """
    dim = args.qk_rope_head_dim
    seqlen = args.max_seq_len
    beta_fast = args.beta_fast # frekans limits
    beta_slow = args.beta_slow
    base = args.rope_theta
    factor = args.rope_factor

    #? Belirtilen rotasyon sayısı için dönme açısı 2π·num_rot eşik değerini geçen boyut indeksini hesaplar
    def find_correction_dim(num_rotations, dim, base, max_seq_len):

        return dim * math.log(max_seq_len / (num_rotations * 2 * math.pi)) / (2 * math.log(base))

    #? Dönme açısının bozulmaya başladığı ve tamamen bozulduğu boyut aralığını belirler
    def find_correction_range(low_rot, high_rot, dim, base, max_seq_len):

        low = math.floor(find_correction_dim(low_rot, dim, base, max_seq_len))
        high = math.ceil(find_correction_dim(high_rot, dim, base, max_seq_len))
        return max(low, 0), min(high, dim-1)

    #? Belirtilen aralıkta [0,1] arasında doğrusal artan bir geçiş (ramp) vektörü oluşturur
    def linear_ramp_factor(min, max, dim):

        if min == max:
            max += 0.001
        linear_func = (torch.arange(dim, dtype=torch.float32) - min) / (max - min)
        ramp_func = torch.clamp(linear_func, 0, 1)
        return ramp_func

    freqs = 1.0 / (base ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
    
    #? Eğer dizi uzunluğu pretraining sınırını aşıyorsa, frekansları yumuşakça düzelt
    if seqlen > args.original_seq_len:
        low, high = find_correction_range(beta_fast, beta_slow, dim, base, args.original_seq_len)
        smooth = 1 - linear_ramp_factor(low, high, dim // 2)
        freqs = freqs / factor * (1 - smooth) + freqs * smooth

    t = torch.arange(seqlen)
    freqs = torch.outer(t, freqs)
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)
    return freqs_cis

def apply_rotary_emb(x: torch.Tensor, freqs_cis: torch.Tensor) -> torch.Tensor:

    assert x.shape[-1] % 2 == 0, "Rotary dim must be divisible by 2!"
    dtype = x.dtype
    x = torch.view_as_complex(x.float().view(*x.shape[:-1], -1, 2))
    freqs_cis = freqs_cis.view(1, x.size(1), 1, x.size(-1))
    y = torch.view_as_real(x * freqs_cis).reshape(*x.shape[:-1], -1)
    return y.to(dtype)

class RMSNorm(nn.Module):

    def __init__(self, dim:int, eps:float=1e-3):
        super().__init__()
        self.dim = dim
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x:torch.Tensor):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True)+ self.eps)

    def forward(self, x:torch.Tensor):
        return self.weight * self._norm(x.float()).type_as(x)

class MLA(nn.Module):

    """
        Öznitelikler (Attributes):
            dim (int): Girdi özelliklerinin boyutu (modelin genel gizli boyutu).
            n_heads (int): Dikkat (attention) başlığı sayısı.
            n_local_heads (int): Dağıtık sistemler için kullanılan lokal attention başlığı sayısı.
            q_lora_rank (int): Query projeksiyonları için düşük-rank (low-rank) LoRA matrislerinin rank değeri.
            kv_lora_rank (int): Key/Value projeksiyonları (C^kv) için düşük-rank LoRA rank değeri.
            qk_nope_head_dim (int): Konumsal bilgi içermeyen query/key projeksiyonlarının boyutu.
            qk_rope_head_dim (int): Rotary positional encoding uygulanan query/key projeksiyonlarının boyutu.
            qk_head_dim (int): Query ve key projeksiyonlarının toplam boyutu.
            v_head_dim (int): Value (değer) projeksiyonlarının boyutu.
            softmax_scale (float): Attention hesaplamalarında softmax’a uygulanan ölçekleme faktörü.
    """
    def __init__(self, args:ModelArgs):
        super().__init__()
        self.dim = args.dim
        self.n_head = args.n_heads
        self.n_local_head = args.n_heads // 1
        self.q_lora_rank = args.q_lora_rank
        self.kv_lora_rank = args.kv_lora_rank
        self.qk_nope_head_dim = args.qk_nope_head_dim
        self.qk_rope_head_dim = args.qk_rope_head_dim
        self.v_head_dim = args.v_head_dim
        self.qk_head_dim = args.qk_nope_head_dim + args.qk_rope_head_dim
        self.isTrain = args.train

        if self.q_lora_rank == 0:
            self.wq = nn.Linear(self.dim, self.n_head * self.qk_head_dim)
        else:
            self.wq_a = nn.Linear(self.dim, self.q_lora_rank) # W_DQ
            self.q_norm = RMSNorm(self.q_lora_rank)
            self.wq_b = nn.Linear(self.q_lora_rank, self.n_head * self.qk_head_dim) # in features: c_t^Q  out features: q_t^C
        
        self.wkv_a = nn.Linear(self.dim, self.kv_lora_rank + self.qk_rope_head_dim) # burada W^DKV ile W_ht^Kr hesaplamaları birliştirildi
        self.kv_norm = RMSNorm(self.kv_lora_rank)
        self.wkv_b = nn.Linear(self.kv_lora_rank, self.n_head * (self.qk_nope_head_dim + self.v_head_dim)) # burada W^uk  x c_t^kv işlemi ile W^uv x c_t^kv işlemleri birleştiriliyor
        self.wo = nn.Linear(self.n_head * self.v_head_dim, self.dim)
        self.softmax_scale = self.qk_head_dim ** -0.5

        if args.max_seq_len > args.original_seq_len:
            mscale = 0.1 * args.mscale * math.log(args.rope_factor) + 1.0
            self.softmax_scale = self.softmax_scale * mscale * mscale


        self.register_buffer('kv_cache', torch.zeros(args.max_batch_size, args.max_seq_len, self.kv_lora_rank), persistent=False) # K/V head'lerinin üretildi latent space
        self.register_buffer('pe_cache', torch.zeros(args.max_batch_size, args.max_seq_len, self.qk_rope_head_dim), persistent=False) # Pozisyon bilgisini bellekte tutma

    def forward(self, x:torch.Tensor, start_pos:int, freqs_cis:torch.Tensor, mask:Optional[torch.Tensor]):
        batch_size, seq_len, _ = x.size()
        end_pos = start_pos + seq_len
        
        if self.q_lora_rank == 0:
            q = self.wq(x)
        else:
            q = self.wq_b(self.q_norm(self.wq_a(x))) # full q_t^c query vector

        q = q.view(batch_size,seq_len, self.n_local_head, self.qk_head_dim) # Divide q into heads
        q_nope, q_pe = torch.split(q, [self.qk_nope_head_dim, self.qk_rope_head_dim], dim=-1) # birlikte hesapladığımız q ve q_rope değerlerini ayırıyoruz
        q_pe = apply_rotary_emb(q_pe, freqs_cis)
        kv = self.wkv_a(x)
        kv, k_pe = torch.split(kv, [self.kv_lora_rank, self.qk_rope_head_dim],dim=-1)
        k_pe = apply_rotary_emb(k_pe.unsqueeze(2), freqs_cis) # k_pe batch_size içermediğinden ona batch_size boyutu ekliyoruz
        

        # deepseek tarzı attention hesaplaması
        wkv_b = self.wkv_b.weight
        wkv_b = wkv_b.view(self.n_local_head, -1, self.kv_lora_rank)
        q_nope = torch.einsum('bshd,hdc->bshc', q_nope, wkv_b[:, :self.qk_nope_head_dim])
        if not self.isTrain:
                    
            self.kv_cache[:batch_size, start_pos:end_pos] = self.kv_norm(kv)
            self.pe_cache[:batch_size, start_pos:end_pos] = k_pe.squeeze(2)

        assert q_nope.shape[-1] == self.kv_cache.shape[-1], "Head dim mismatch between q_nope and kv_cache" 
        kv = self.kv_cache[:batch_size, :end_pos].unsqueeze(2)  # -> [B, T, 1, R]
        pe = self.pe_cache[:batch_size, :end_pos].unsqueeze(2)  # -> [B, T, 1, R]
        scores = (
             torch.einsum('bshr,bthr->bsht', q_nope, kv) +
             torch.einsum('bshr,bthr->bsht', q_pe, pe)
            ) * self.softmax_scale

        if mask is None and end_pos > 1:
            mask = torch.full((end_pos, end_pos), float('-inf'), device=x.device).triu(1)

        scores = scores.softmax(dim=-1, dtype=torch.float32).type_as(x)

        x = torch.einsum('bsht,btc->bshc',scores, self.kv_cache[:batch_size, :end_pos])
        x = torch.einsum('bshc,hdc->bshd',x,wkv_b[:,-self.v_head_dim:])
        x = self.wo(x.flatten(2))
        return x

In [15]:
cfg = ModelArgs()
mla = MLA(cfg)

In [23]:
a = torch.rand(4,512,cfg.dim)

In [24]:
freqs = precompute_freqs_cis(cfg)
print(freqs.shape)


torch.Size([256, 16])


In [25]:
b = mla(a,0,freqs,mask=None)

RuntimeError: shape '[1, 512, 1, 16]' is invalid for input of size 4096